In [1]:
import pandas as pd
import numpy as np
from typing import NewType
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from graphviz import Digraph

### Iris dataset

In [37]:
iris = datasets.load_iris()
X = iris.data[:, :3] 
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

### HW dataset

In [2]:
data = pd.read_csv("../datasets/01_homework_dataset.csv")
X = np.array(data.loc[:, data.columns[:3]])
y = np.array(data.loc[:, data.columns[3]])
x_a = np.array([4.1, 0.1, 2.2])
x_b = np.array([6.1, 0.4, 1.3])

In [3]:
data

,x1,x2,x3,z
0,5.5,0.5,4.5,2
1,7.4,1.1,3.6,0
2,5.9,0.2,3.4,2
3,9.9,0.1,0.8,0
4,6.9,-0.1,0.6,2
5,6.8,-0.3,5.1,2
6,4.1,0.3,5.1,1
7,1.3,-0.2,1.8,1
8,4.5,0.4,2.0,0
9,0.5,0.0,2.3,1


In [4]:
class Node:
    def __init__(self):
        self.prediction = None
        self.gini = None
        self.j_ast = None
        self.t_ast = None
        self.right = None
        self.left = None
        
    def __repr__(self):
        return f"Predictions: {self.prediction} \n Gini: {self.gini}\n J*: {self.j_ast}\n  Trashhold: {self.t_ast} \n Right: {self.right} \n Left: {self.left} \n"

In [5]:
NodeType = NewType("NodeType", Node)

In [12]:
class Tree:
    def __init__(self):
        self.node = None
    
    @staticmethod
    def gini(probs) -> float:
        return 1 - np.sum(probs**2)
    
    @staticmethod
    def get_probabilities(y: np.array) -> np.array:
        _, counts = np.unique(y, return_counts=True)
        return np.true_divide(counts, np.sum(counts))
    
    @staticmethod
    def split(D: np.array, y: np.array) -> tuple:
        n, m = D.shape[0], D.shape[1]
        gini_min, j_ast, t_ast, D_l, D_r , y_l, y_r = [float("inf")] + [None]*6
        for j in range(m):
            for i in range(n):
                trashhold = D[i,j]
                indexes = D[:,j] < trashhold
                probs_le, probs_ge = Tree.get_probabilities(y[indexes]), Tree.get_probabilities(y[~indexes])
                gini_le, gini_ge = Tree.gini(probs_le), Tree.gini(probs_ge)
                if (gini_le + gini_ge) < gini_min:
                    gini_min = gini_le + gini_ge
                    j_ast = j
                    t_ast = trashhold
                    D_l = D[indexes]
                    D_r = D[~indexes]
                    y_l = y[indexes]
                    y_r = y[~indexes]
        return j_ast, t_ast, D_l, D_r, y_l, y_r, gini_min
        
    @staticmethod
    def fitTree(node: NodeType, D: np.array, y: np.array, depth:int) -> NodeType:
        node.prediction = Counter(y)
        if (depth == 2) or (len(node.prediction) == 1):
            return node
        else:
            j_ast, t_ast, D_l, D_r, y_l, y_r, gini_min = Tree.split(D, y)
            node.gini = gini_min
            node.j_ast = j_ast
            node.t_ast = t_ast
            node.left = Tree.fitTree(Node(), D_l, y_l, depth + 1)
            node.right = Tree.fitTree(Node(), D_r, y_r, depth + 1)
            return node
    
    def fit(self, X, y):
        self.node = Node()
        self.node = self.fitTree(self.node, X, y, depth=0)
        
    @staticmethod
    def predict_element(node, x_new):
        if (node.right == None) & (node.left == None):
            return max(node.prediction, key=node.prediction.get)
        if x_new[node.j_ast] <= node.t_ast:
            return Tree.predict_element(node.left, x_new)
        elif x_new[node.j_ast] > node.t_ast:
            return Tree.predict_element(node.right, x_new)
    
    def predict(self, X_new):
        result = np.array([])
        for x_new in X_new:
            result = np.append(result, Tree.predict_element(self.node, x_new))
        return result

    def visualize_node(self, node, node_number):
        if not (node.left is None and node.right is None):
            self.g.node(str(int(node_number)), 
                        label=r'x_{} {} {} \n G: {} \n {}'.\
                                format(node.j_ast, '<', node.t_ast, node.gini, node.prediction))
            self.visualize_node(node.left, 2 * node_number + 1)
            self.visualize_node(node.right, 2 * node_number + 2)
            if node_number != 0:
                self.g.edge(str((node_number - 1) // 2), str(int(node_number)))
        else:
            self.g.node(str(int(node_number)), label=r'{}'.format(node.prediction))
            self.g.edge(str((node_number - 1) // 2), str(int(node_number)))

    def export_to_graphviz(self, filename='test-output/round-table.gv'):
        self.g = Digraph('DT', format='png')
        node = self.node
        self.visualize_node(node, 0)
        self.g.render(filename, view=True) 

### Problem 2

In [13]:
tree = Tree()

In [14]:
tree.fit(X,y)

In [15]:
print(tree.node)

Predictions: Counter({1: 6, 0: 5, 2: 4}) 
 Gini: 0.49382716049382713
 J*: 0
  Trashhold: 4.5 
 Right: Predictions: Counter({0: 5, 2: 4}) 
 Gini: 0.40816326530612246
 J*: 2
  Trashhold: 4.5 
 Right: Predictions: Counter({2: 2}) 
 Gini: None
 J*: None
  Trashhold: None 
 Right: None 
 Left: None 
 
 Left: Predictions: Counter({0: 5, 2: 2}) 
 Gini: None
 J*: None
  Trashhold: None 
 Right: None 
 Left: None 
 
 
 Left: Predictions: Counter({1: 6}) 
 Gini: None
 J*: None
  Trashhold: None 
 Right: None 
 Left: None 
 



In [ ]:
tree.export_to_graphviz()

In [ ]:
tree.predict(np.array([x_a, x_b]))

### Validation iris data

In [46]:
tree = Tree()

In [ ]:
tree.fit(X_train, y_train)

In [48]:
y_predicted = tree.predict(X_test)
print(f"Accuracy {accuracy_score(y_test, y_predicted)*100}%")

Accuracy 89.47368421052632%
